In [ ]:
#take shannon matlab & put into netcdf
from scipy.io import loadmat

import h5py
import xarray as xr
import numpy as np

#PLOTTING
import cartopy
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.colorbar import Colorbar
import matplotlib.ticker as mticker

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

#resampling to grid
from pyresample.geometry import AreaDefinition
from pyresample.geometry import GridDefinition
from pyresample import image, geometry, load_area, save_quicklook, SwathDefinition, area_def2basemap
from pyresample.kd_tree import resample_nearest
from pyresample.utils import check_and_wrap
from scipy import spatial

import glob

#where to find the data
adir_data= 'f:/data/project_data/fluxsat/orbit/'
adir_figs= 'f:/data/project_data/fluxsat/figures/'

# add land mask
#get bathymetry from ETOPO1
fname_topo = 'F:/data/topo/ETOPO1_Ice_g_gmt4.grd'
ds = xr.open_dataset(fname_topo)
ds_topo = ds.rename_dims({'x':'lon','y':'lat'}).rename({'x':'lon','y':'lat'})
tem = ds_topo.attrs
ds_topo = ds_topo.rename({'z':'etopo_depth'})
ds_topo.etopo_depth.attrs=tem
_, index = np.unique(ds_topo['lon'], return_index=True)
ds_topo = ds_topo.isel(lon=index)
_, index = np.unique(ds_topo['lat'], return_index=True)
ds_topo = ds_topo.isel(lat=index)

In [ ]:
#get all filenames
filenames = glob.glob(adir_data+'FS*.mat')
filenames

In [ ]:
#read in data from all 3 files, put into xarray dataset
#change UTCtime from days since 0000/1/1 to datetime64 object
#make utctime the coordinate
ds_sv=[]
for fname in filenames:
    f = h5py.File(fname,mode='r') #open file 
    struArray = f['rad']
    ilen =  struArray['SClat']['data'].shape[0]
    da = xr.DataArray(None, coords=[np.arange(ilen)], dims=['iob'])
    ds = xr.Dataset({'test':xr.DataArray(None, coords=[np.arange(ilen)], dims=['iob'])})
    for v in struArray:
        data = struArray[v]['data']
        da = xr.DataArray(data[:,0], coords=[np.arange(ilen)], dims=['iob'])
        ds[v]=da
    ds['UTCtime'] = np.datetime64('0000-01-01')+ds.UTCtime.astype('timedelta64[D]')+((ds.UTCtime-ds.UTCtime[0])*24*60*60*1e9).astype('timedelta64[ns]')
    ds = ds.drop('test')
    ds = ds.swap_dims({'iob':'UTCtime'}).reset_coords()
    ds_sv.append(ds)
ds = xr.concat(ds_sv, dim='UTCtime')
#ds['UTCtime']=np.datetime64('0000-01-01')+ds.UTCtime.astype('int').data.astype('timedelta64[D]')

In [ ]:
#create 1 day of data dataarray in xarray
dylist = ['2018-11-28','2018-11-29','2018-11-30']
da_sv = []
for dnum,dstr in enumerate(dylist):
    dy = ds.sel(UTCtime=slice(dylist[dnum],dylist[dnum]))
    for iorbit in range(16):
        i1=iorbit*2777296
        i2=(iorbit+1)*2777296
        i2= [dy.blon.size-1 if i2>dy.blon.size else i2]
        i2=i2[0]
        dy2 = dy.sel(UTCtime=slice(dy.UTCtime[i1],dy.UTCtime[i2]))       
        x = dy2.blon.data
        y = dy2.blat.data
        z = dy2.blat.data*0+1
        lons,lats,data = x,y,z 
        lons,lats = check_and_wrap(lons, lats)
        swath_def = SwathDefinition(lons, lats)
        grid_def_lons, grid_def_lats = np.arange(-180,180,0.1), np.arange(-90,90,0.1)
        grid_mesh_lons,grid_mesh_lats = np.meshgrid(grid_def_lons,grid_def_lats)
        grid_def = GridDefinition(lons=grid_mesh_lons,lats=grid_mesh_lats)
        result1 = resample_nearest(swath_def, data, grid_def, radius_of_influence=20000, fill_value=None)
        rr=np.expand_dims(result1,axis=2)
        iorbit_num = iorbit+dnum*16
        da = xr.DataArray(rr,name='sec',
                          coords={'lat':grid_def_lats,'lon':grid_def_lons,'orbit':[iorbit_num]},
                          dims=('lat','lon','orbit'))
        da_sv.append(da)
da2 = xr.concat(da_sv, dim='orbit')


In [ ]:
#ds = ds.drop({'SCinc','iob','SCalt','belev','bhorz','binc','scanang','SClat','SClon'})
fname_out = adir_data + '3dys_butterfly_obs.nc'
ds.to_netcdf(fname_out)
fname_out = adir_data + '3dys_butterfly_grid.nc'
da2.to_netcdf(fname_out)


In [ ]:
#breakup
fname_out = adir_data + '3dys_butterfly_grid.nc'
ds = xr.open_dataset(fname_out)
fname_out = adir_data + '3dys_butterfly_gridA.nc'
ds.sec[:,:,0:16].to_netcdf(fname_out,encoding={"sec": {"dtype": "int8", "scale_factor": 0.1, "zlib": True, "complevel": 9}})
fname_out = adir_data + '3dys_butterfly_gridB.nc'
ds.sec[:,:,16:32].to_netcdf(fname_out,encoding={"sec": {"dtype": "int8", "scale_factor": 0.1, "zlib": True, "complevel": 9}})
fname_out = adir_data + '3dys_butterfly_gridC.nc'
ds.sec[:,:,32:].to_netcdf(fname_out,encoding={"sec": {"dtype": "int8", "scale_factor": 0.1, "zlib": True, "complevel": 9}})



In [ ]:
tem = xr.open_dataset(fname_out)
tem.close()
tem

In [ ]:
tem.sec[:,:,0].plot()

In [ ]:
# CAC data
from scipy.io import loadmat
#load data
x = loadmat(adir_data + 'clayson_fluxes.mat')
#put into xarray dataarray
da_lo = xr.DataArray(x['lhf1deg'], 
                     coords={'latlo':x['lat1deg'][0],'lonlo':x['lon1deg'][0]}, 
                     dims=['lonlo','latlo']).T
da_hi = xr.DataArray(x['lhfhires'], 
                     coords={'lat':x['latitude'][:,0],'lon':x['longitude'][:,0]}, 
                     dims=['lon','lat']).T
#put into xarray dataset & write to netcdf file
ds = xr.Dataset({'lo':da_lo,'hi':da_hi})
ds.to_netcdf(adir_data + 'clayson_fluxes.nc')

In [ ]:
da_hi.plot()

In [ ]:
# shannons different resolution fluxes
#fname = adir_data + 'shannon_flux_res.mat'
# CAC data
from scipy.io import loadmat
#load data
x = loadmat(adir_data + 'shannon_flux_res.mat')
lat1 = x['XI'][1,:]
lon1 = x['YI'][:,1]
hi = x['hires']
lo = x['lores']
#put into xarray dataarray
da_lo = xr.DataArray(lo, 
                     coords={'lat':lat1,'lon':lon1,'res':[0,1,2,3]}, 
                     dims=['lon','lat','res']).T
da_hi = xr.DataArray(hi, 
                     coords={'lat':lat1,'lon':lon1,'res':[0,1,2]}, 
                     dims=['lon','lat','res']).T
#put into xarray dataset & write to netcdf file
ds = xr.Dataset({'lo':da_lo,'hi':da_hi})
ds.to_netcdf(adir_data + 'clayson_fluxes_hilo.nc')



In [ ]:
ds.hi[0,:,:].sel(lon=slice(120,180),lat=slice(10,50)).plot(cmap='jet')

In [ ]:
ds.hi[2,:,:].sel(lon=slice(120,180),lat=slice(10,50)).plot(cmap='jet')